In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import models, layers, regularizers

In [ ]:
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1)
)

config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

In [ ]:
data_train = np.load('<path_to_file>/train_data.npy', allow_pickle=True)
train_labels = np.load('<path_to_file>/train_labels.npy', allow_pickle=True)

data_val = np.load('<path_to_file>/val_data.npy', allow_pickle=True)
val_labels = np.load('<path_to_file>/val_labels.npy', allow_pickle=True)

data_test = np.load('<path_to_file>/test_data.npy', allow_pickle=True)
test_labels = np.load('<path_to_file>/test_labels.npy', allow_pickle=True)

# Check the shapes of all datasets
data_train.shape, train_labels.shape, data_val.shape, val_labels.shape, data_test.shape, test_labels.shape

In [ ]:
# Compute mean and std from training data (per-channel normalization)
mean = data_train.mean(axis=(0, 1, 2), keepdims=True)
std_dev = data_train.std(axis=(0, 1, 2), keepdims=True)

# Normalize all datasets using training statistics
data_train = (data_train - mean) / std_dev
data_val = (data_val - mean) / std_dev
data_test = (data_test - mean) / std_dev

# Verify shapes
print("Shapes after normalization:")
print("Train:", data_train.shape, "Labels:", train_labels.shape)
print("Val:", data_val.shape, "Labels:", val_labels.shape)
print("Test:", data_test.shape, "Labels:", test_labels.shape)

In [ ]:
le = LabelEncoder()

y_train = le.fit_transform(train_labels)
y_val = le.transform(val_labels)
y_test = le.transform(test_labels)

print("Class mapping:", le.classes_)

In [ ]:
for class_name, encoded_label in zip(le.classes_, range(len(le.classes_))):
    
    print(f"Original: '{class_name}' \t-> Encoded: {encoded_label}")

In [ ]:
# Add channel dimension to data
data_train = data_train[..., np.newaxis]
data_val = data_val[..., np.newaxis]
data_test = data_test[..., np.newaxis]

data_train.shape ,data_val.shape, data_test.shape

In [ ]:
def augment_data(sample, label):
    
    # Ensure both are the same type
    random_scale = tf.cast(tf.random.uniform([], 0.9, 1.1), dtype=sample.dtype)
    sample = sample * random_scale  # Random scale between 0.9 and 1.1
    
    return sample, label

In [ ]:
# Convert all data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((data_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((data_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((data_test, y_test))

# Configure datasets
batch_size = 4

train_dataset = train_dataset.shuffle(buffer_size=100).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
#tf.random.set_seed(1234)

input_shape = (32, 32, 23, 1)

model = models.Sequential([
    
    #1st Conv Layer
    layers.Conv3D(32, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation='relu',
                  kernel_regularizer=regularizers.l2(0.01), input_shape=input_shape),
    layers.AveragePooling3D(pool_size=(2, 2, 2)),
  
     
    #2nd Conv Layer
    layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),

    #3rd Conv Layer
    layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation='relu',
                  kernel_regularizer=regularizers.l2(0.0001)),
    layers.AveragePooling3D(pool_size=(2, 2, 2)),

    layers.Flatten(),
    
    #1st FC layer
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),  # Dropout with a rate of 0.5

    #2nd FC layer
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    layers.Dense(3, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = '3DCNN_Avacado_PCA_model.h5',            # Path to save the best model
    monitor = 'val_accuracy',                   # Metric to monitor
    save_best_only = True,                 # Save only the best model
    save_weights_only = False,             # Save the entire model (architecture + weights)
    mode = 'max',                          # Mode 'max' because we want to maximize accuracy
    verbose = 1
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'loss',
    patience = 15,
    restore_best_weights = True
)

# Train the model with the callbacks
history = model.fit(
    train_dataset,
    epochs = 100,
    validation_data = val_dataset,
    callbacks = [checkpoint_callback, early_stopping_callback]
)

In [ ]:
# Plot training & validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Get true labels and predictions
test_labels = np.concatenate([y for x, y in test_dataset], axis=0)
predictions = np.argmax(model.predict(test_dataset), axis=-1)

In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(test_labels, predictions))

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[str(i) for i in range(2)], yticklabels=[str(i) for i in range(2)])

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

plt.show()